## 加载官方提供的 ner_dict到jieba

In [ ]:
import jieba
file_name='./data_v2/demo_entity.txt'
jieba.load_userdict(file_name)

In [5]:
import json
from tqdm import tqdm

## 加载官方的ner_dict 当作候选集

In [52]:
ner_dic_file='./data_v2/demo_entity.txt'
ner_dic=set()
with open(ner_dic_file, 'r', encoding='utf-8') as rf:
    for item in rf:
        item=item.strip()
        if item!='':
            ner_dic.add(item.strip())

In [54]:
len(ner_dic)

321421

##  tiran_demo
### in_file 输入训练集
### out_file 输出处理完的文件

In [ ]:
total=0    
unk_num=0
unk_word=[]
max_l=15   


in_file='./coreEntityEmotion_train_tt.txt'
out_file='./data/model.train'
with open(out_file, 'w', encoding='utf-8') as wf:
    with open(in_file, 'r', encoding='utf-8') as rf:
        for line in tqdm(rf):
            line=json.loads(line)
            
            total+=len(line['coreEntityEmotions'])
            
            
            #先将实体大写变小写
            for entity in line['coreEntityEmotions']:
                entity['entity']=entity['entity'].lower()
            
            #文章分词========================
            for entity in line['coreEntityEmotions']:
                jieba.add_word(entity['entity'], freq=1e10)
                ner_dic.add(entity['entity'])
            

            text=jieba.lcut((line['title']+'\t'+line['content']).lower())[:550]
            text=text+['<end>']
            
            #最大匹配算法======================
            
            text_temp=[]
            i=0
            j=1
            while i<len(text):
                best_word=text[i]
                a_word=text[i]
                if best_word=='《':
                    text_temp.append(best_word)
                    i+=1
                    j=i+1
                else:
                    while j<len(text):
                        if len(a_word)>max_l:
                            break
                        a_word=a_word+text[j]
                        if a_word in ner_dic:
                            zuhe_num+=1
                            best_word=a_word
                            zuhe_word.append(best_word)
                            i=j
                        j+=1
                    text_temp.append(best_word)
                    i+=1
                    j=i+1
            text=text_temp
            line['text']=text
            
            #最大匹配算法=============================
            
            #纪录核心实体位置=============================
            
            temp=[]
            for entity in line['coreEntityEmotions']:
                if entity['entity'] in text :
                    temp.append([text.index(entity['entity']), entity])
                else:
                    unk_num+=1
                    unk_word.append(entity['entity'])
            if len(temp) ==0:
                continue
                
            temp.sort(key=lambda x:x[0])
            line['true_Entity']=temp
            #纪录核心实体位置=============================
            
            #mask,对在候选集的单词置1，其他置0=============================
    
            mask=[]
            for item in text:
                if item in ner_dic or item=='<end>':
                    mask.append(1)
                else:
                    mask.append(0)
            line['mask']=mask

            
            #mask,对在候选集的单词置1，其他置0=============================
            
            wf.write(json.dumps(line, ensure_ascii=False)+'\n')

## test_demo

In [38]:
in_file='./data/coreEntityEmotion_test_stage1.txt'
out_file='./data/model.test.raw'
max_l=15
id0=-1

zuhe_num=0
zuhe_word=[]
with open(out_file, 'w', encoding='utf-8') as wf:
    with open(in_file, 'r', encoding='utf-8') as rf:
        for line in tqdm(rf):
            id0+=1
            if id0==3000:
                wf.close()
                break
            line=json.loads(line)
            

            text=jieba.lcut((line['title']+'\t'+line['content']).lower())[:550]
            text=text+['<end>']
            
            text_temp=[]
            i=0
            j=1
            while i<len(text):
                best_word=text[i]
                a_word=text[i]
                if best_word=='《':
                    text_temp.append(best_word)
                    i+=1
                    j=i+1
                else:
                    while j<len(text):
                        if len(a_word)>max_l:
                            break
                        a_word=a_word+text[j]
                        if a_word in ner_dic:
                            zuhe_num+=1
                            best_word=a_word
                            zuhe_word.append(best_word)
                            i=j
                        j+=1
                    text_temp.append(best_word)
                    i+=1
                    j=i+1
            text=text_temp
            
            
            line['text']=text
            
            temp=[[0,{'emotion': 'POS', 'entity': 'KKK'}], ]
            line['true_Entity']=temp
            
            mask=[]

            for item in text:
                if item in ner_dic or item=='<end>':
                    mask.append(1)
                else:
                    mask.append(0)
            line['mask']=mask
                
            
            wf.write(json.dumps(line, ensure_ascii=False)+'\n')





0it [00:00, ?it/s]



5it [00:00, 48.85it/s]



14it [00:00, 55.20it/s]



23it [00:00, 61.34it/s]



33it [00:00, 68.32it/s]



41it [00:00, 67.60it/s]



51it [00:00, 73.50it/s]



65it [00:00, 85.35it/s]



78it [00:00, 93.94it/s]



89it [00:00, 94.52it/s]



99it [00:01, 92.77it/s]



110it [00:01, 96.54it/s]



121it [00:01, 99.01it/s]



132it [00:01, 101.82it/s]



143it [00:01, 103.73it/s]



154it [00:01, 97.47it/s] 



166it [00:01, 100.87it/s]



181it [00:01, 110.53it/s]



193it [00:01, 104.10it/s]



204it [00:02, 92.87it/s] 



214it [00:02, 81.95it/s]



223it [00:02, 83.44it/s]



234it [00:02, 89.49it/s]



244it [00:02, 92.30it/s]



257it [00:02, 97.93it/s]



268it [00:02, 98.09it/s]



281it [00:02, 104.70it/s]



292it [00:03, 99.55it/s] 



303it [00:03, 97.64it/s]



314it [00:03, 100.68it/s]



325it [00:03, 96.87it/s] 



336it [00:03, 99.96it/s]



348it [00:03, 104.67it/s]



359it [00:03, 105.69it/s]



370it [00:03, 106.35it/s]



383it [00:03, 110.2